In [71]:
from ds_utils.imports import *

# Prework

Following this fast.ai [lesson](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson4.ipynb) and [video](https://www.youtube.com/watch?v=V2h3IOBDvrA&feature=youtu.be&t=5761).

In [84]:
# get data
# !wget -O data/ml-latest-small.zip http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !unzip data/ml-latest-small.zip

# Preprocessing

In [85]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

In [86]:
ratings.shape

(100004, 4)

In [87]:
ratings.sample(5)

,userId,movieId,rating,timestamp
637,8,2502,5.0,1154464889
31338,225,253,5.0,845565809
73595,514,507,4.0,853896152
24276,176,595,4.0,1341001312
79763,547,6584,2.5,1200868443


In [88]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [89]:
userid2idx = {o: i for i, o in enumerate(users)}
movieid2idx = {o: i for i, o in enumerate(movies)}

In [90]:
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])

In [91]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

In [92]:
n_factors = 50

In [93]:
np.random.seed = 42

In [94]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

# Dot product

In [108]:
user_in = keras.layers.Input(shape=(1, ), dtype='int64', name='user_in')

In [109]:
u = keras.layers.Embedding(
    input_dim=n_users,
    output_dim=n_factors,
    input_length=1,
    embeddings_regularizer=keras.regularizers.l2(l=1e-4))(user_in)

In [111]:
movie_in = keras.layers.Input(shape=(1, ), dtype='int64', name='movie_in')

In [112]:
m = keras.layers.Embedding(
    input_dim=n_movies,
    output_dim=n_factors,
    input_length=1,
    embeddings_regularizer=keras.regularizers.l2(l=1e-4))(movie_in)

In [113]:
x = keras.layers.merge([u, m], mode='dot')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [114]:
x = keras.layers.Flatten()(x)

In [115]:
model = keras.models.Model(inputs=[user_in, movie_in], outputs=x)

In [116]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='mse')

In [117]:
model.fit(x=[trn.userId, trn.movieId],
          y=trn.rating,
          batch_size=64,
          epochs=1,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79907 samples, validate on 20097 samples
Epoch 1/1
79907/79907 [==============================] - 8s - loss: 10.1298 - val_loss: 4.4034

In [67]:
model.optimizer.lr = 0.01

In [118]:
model.fit(
    x=[trn.userId, trn.movieId],
    y=trn.rating,
    batch_size=64,
    epochs=3,
    validation_data=([val.userId, val.movieId], val.rating), )

Train on 79907 samples, validate on 20097 samples
Epoch 1/3
79907/79907 [==============================] - 11s - loss: 3.1727 - val_loss: 2.8120

In [119]:
model.optimizer.lr = 0.001

In [120]:
model.fit(x=[trn.userId, trn.movieId],
          y=trn.rating,
          batch_size=64,
          epochs=6,
          validation_data=([val.userId, val.movieId], val.rating),
          shuffle=False)

Train on 79907 samples, validate on 20097 samples
Epoch 1/6
79907/79907 [==============================] - 10s - loss: 2.2667 - val_loss: 2.7215

# Bias

In [130]:
user_bias = keras.layers.Embedding(input_dim=n_users, output_dim=1, input_length=1)(user_in)
user_bias = keras.layers.Flatten()(user_bias)

In [131]:
movie_bias = keras.layers.Embedding(input_dim=n_movies, output_dim=1, input_length=1)(movie_in)
movie_bias = keras.layers.Flatten()(movie_bias)

In [132]:
x = keras.layers.merge([u, m], mode='dot')
x = keras.layers.Flatten()(x)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [133]:
x = keras.layers.merge([x, user_bias], mode='sum')
x = keras.layers.merge([x, movie_bias], mode='sum')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [151]:
model = keras.models.Model(inputs=[user_in, movie_in], outputs=x)

In [152]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='mse')

In [158]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_8 (Embedding)          (None, 1, 50)         33550                                        
____________________________________________________________________________________________________
embedding_9 (Embedding)          (None, 1, 50)         453300                                       
___________________________________________________________________________________________

In [153]:
model.fit(x=[trn.userId, trn.movieId], y=trn.rating, batch_size=64, validation_data=([val.userId, val.movieId], val.rating))

Train on 79907 samples, validate on 20097 samples
Epoch 1/1
79907/79907 [==============================] - 13s - loss: 0.8260 - val_loss: 1.0911

In [154]:
model.optimizer.lr=0.01

In [155]:
model.fit(x=[trn.userId, trn.movieId], y=trn.rating, batch_size=64, epochs=10, validation_data=([val.userId, val.movieId], val.rating))

Train on 79907 samples, validate on 20097 samples
Epoch 1/10
79907/79907 [==============================] - 12s - loss: 0.7332 - val_loss: 1.0819

In [156]:
model.optimizer.lr=0.001

In [157]:
model.fit(x=[trn.userId, trn.movieId], y=trn.rating, batch_size=64, epochs=5, validation_data=([val.userId, val.movieId], val.rating))

Train on 79907 samples, validate on 20097 samples
Epoch 1/5
79907/79907 [==============================] - 13s - loss: 0.5616 - val_loss: 1.0311

# scratch